In [1]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from dotenv import load_dotenv
from langchain.llms import OpenAI
import os

#### OpenAI LLM & API key setup

In [2]:
 dotenv_path = "C:/Users/burha/OneDrive/Desktop/Career/api_key.env"
load_dotenv(dotenv_path)

True

In [3]:
api_key = os.getenv("api_key")
llm = OpenAI(openai_api_key=api_key, temperature=0.2)

#### Connect with database

In [4]:
db_user = "root"
db_password = "root"
db_host = "localhost"
db_name = "classicmodels"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE customers (
	`customerNumber` INTEGER NOT NULL, 
	`customerName` VARCHAR(50) NOT NULL, 
	`contactLastName` VARCHAR(50) NOT NULL, 
	`contactFirstName` VARCHAR(50) NOT NULL, 
	phone VARCHAR(50) NOT NULL, 
	`addressLine1` VARCHAR(50) NOT NULL, 
	`addressLine2` VARCHAR(50), 
	city VARCHAR(50) NOT NULL, 
	state VARCHAR(50), 
	`postalCode` VARCHAR(15), 
	country VARCHAR(50) NOT NULL, 
	`salesRepEmployeeNumber` INTEGER, 
	`creditLimit` DECIMAL(10, 2), 
	PRIMARY KEY (`customerNumber`), 
	CONSTRAINT customers_ibfk_1 FOREIGN KEY(`salesRepEmployeeNumber`) REFERENCES employees (`employeeNumber`)
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from customers table:
customerNumber	customerName	contactLastName	contactFirstName	phone	addressLine1	addressLine2	city	state	postalCode	country	salesRepEmployeeNumber	creditLimit
103	Atelier graphique	Schmitt	Carine 	40.32.2555	54, rue Royale	None	Nantes	None	44000	France	1370	21000.00
112	Signal Gift Stores	King	Je

#### Few shot learning

In [7]:
few_shots = [
    {'Question' : "Who reports to William Patterson?",
     'SQLQuery' : "SELECT CONCAT(reports.firstname, ' ', reports.lastname) AS Employee FROM Employees boss JOIN Employees reports ON boss.employeeNumber = reports.reportsTo WHERE boss.firstName = 'William' and boss.lastName = 'Patterson';",
     'SQLResult': "Result of the SQL query",
     'Answer' : "'Andy Fixter', 'Peter Marsh', 'Tom King'"},
    {'Question': "Compute the commission for each sales representative, assuming the commission is 5% of the value of an order. Sort by employee last name and first name",
     'SQLQuery': "SELECT CONCAT(firstName, ' ',lastName) AS Name, FORMAT(.05 * SUM(quantityOrdered * priceEach),0) AS Commission FROM Employees JOIN Customers ON Employees.employeeNumber = Customers.salesRepEmployeeNumber JOIN Orders ON Customers.customerNumber = Orders.customerNumber JOIN OrderDetails ON Orders.orderNumber = OrderDetails.orderNumber GROUP BY employeeNumber ORDER BY lastName, firstName ASC;",
     'SQLResult': "Result of the SQL query",
     'Answer': "'Loui Bondur, 28,474', 'Larry Bott, 36,605', 'Pamela Castillo, 43,411', 'Julie Firrelli, 19,333','Andy Fixter, 28,129', 'Martin Gerard, 19,374'......"},
    {'Question': "What are the top 3 products by quantity of on hand for products listed on 'On Hold' orders?" ,
     'SQLQuery' : "SELECT productName, FORMAT(quantityInStock,0) AS `Quantity in stock` FROM OrderDetails JOIN Orders ON Orders.orderNumber = OrderDetails.orderNumber JOIN Products on OrderDetails.productCode = Products.productCode WHERE status = 'On Hold' order by quantityInStock desc limit 3;",
     'SQLResult': "Result of the SQL query",
     'Answer': "'America West Airlines B757-200, 9,653', '2002 Chevy Corvette, 9,446', '1912 Ford Model T Delivery Wagon, 9,173'"} ,
     {'Question' : "What is the value of orders shipped in August 2004?" ,
      'SQLQuery': "SELECT FORMAT(SUM(quantityOrdered*priceEach),0) as orderValue FROM Orders JOIN OrderDetails ON Orders.orderNumber = OrderDetails. orderNumber AND YEAR(orderDate) = 2004 AND MONTH(orderDate) = 8;",
      'SQLResult': "Result of the SQL query",
      'Answer' : '419,327'},
    {'Question' : "What is the average percentage markup of the MSRP on buyPrice rounded to the nearest deicmal?" ,
      'SQLQuery': "SELECT ROUND(AVG(MSRP/buyPrice)*100 - 100) from Products;",
      'SQLResult': "Result of the SQL query",
      'Answer' : '89'},
    {'Question' : "Who are the employees in Boston?" ,
      'SQLQuery': "SELECT firstName, lastName FROM Employees JOIN Offices ON Employees.officeCode = Offices.officeCode AND city = 'Boston';",
      'SQLResult': "Result of the SQL query",
      'Answer' : '"Julie Firrelli", "Steve Patterson"'},
    {'Question' : "List the names of the top 3 products (by inventory) sold at less than 80% of the MSRP." ,
      'SQLQuery': "SELECT DISTINCT(productName), priceEach, MSRP, FORMAT(quantityInStock,0) AS `Quantity in stock` FROM OrderDetails JOIN Products ON OrderDetails.productCode = Products.productCode AND priceEach < .8*MSRP;",
      'SQLResult': "Result of the SQL query",
      'Answer' : '1976 Ford Gran Torino	117.59	146.99	9127, 1965 Aston Martin DB5	99.55	124.44	9042 American Airlines: MD-11S	59.22	74.03	8820'},
    {'Question' : "What are the top 3 most profitable product lines" ,
      'SQLQuery': "SELECT productLine AS `Product line`, FORMAT(SUM(quantityOrdered*(priceEach -buyPrice)),0) AS Profit FROM Products JOIN OrderDetails ON Products.productCode = OrderDetails.productCode GROUP BY productLine ORDER BY SUM(quantityOrdered*(priceEach -buyPrice)) DESC LIMIT 3;",
      'SQLResult': "Result of the SQL query",
      'Answer' : 'Classic Cars, Vintage Cars, and Motorcycles'}
    

    
]

### Creating Semantic Similarity Based example selector

- create embedding on the few_shots
- Store the embeddings in Chroma DB
- Retrieve the the top most Semantically close example from the vector store

In [8]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [9]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

to_vectorize = [" ".join(example.values()) for example in few_shots]

In [10]:
to_vectorize

["Who reports to William Patterson? SELECT CONCAT(reports.firstname, ' ', reports.lastname) AS Employee FROM Employees boss JOIN Employees reports ON boss.employeeNumber = reports.reportsTo WHERE boss.firstName = 'William' and boss.lastName = 'Patterson'; Result of the SQL query 'Andy Fixter', 'Peter Marsh', 'Tom King'",
 "Compute the commission for each sales representative, assuming the commission is 5% of the value of an order. Sort by employee last name and first name SELECT CONCAT(firstName, ' ',lastName) AS Name, FORMAT(.05 * SUM(quantityOrdered * priceEach),0) AS Commission FROM Employees JOIN Customers ON Employees.employeeNumber = Customers.salesRepEmployeeNumber JOIN Orders ON Customers.customerNumber = Orders.customerNumber JOIN OrderDetails ON Orders.orderNumber = OrderDetails.orderNumber GROUP BY employeeNumber ORDER BY lastName, firstName ASC; Result of the SQL query 'Loui Bondur, 28,474', 'Larry Bott, 36,605', 'Pamela Castillo, 43,411', 'Julie Firrelli, 19,333','Andy Fix

In [11]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [12]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "Who reports to Mary Patterson?"})

[{'Answer': "'Andy Fixter', 'Peter Marsh', 'Tom King'",
  'Question': 'Who reports to William Patterson?',
  'SQLQuery': "SELECT CONCAT(reports.firstname, ' ', reports.lastname) AS Employee FROM Employees boss JOIN Employees reports ON boss.employeeNumber = reports.reportsTo WHERE boss.firstName = 'William' and boss.lastName = 'Patterson';",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '"Julie Firrelli", "Steve Patterson"',
  'Question': 'Who are the employees in Boston?',
  'SQLQuery': "SELECT firstName, lastName FROM Employees JOIN Offices ON Employees.officeCode = Offices.officeCode AND city = 'Boston';",
  'SQLResult': 'Result of the SQL query'}]

In [13]:
example_selector.select_examples({"Question": "Report the number of orders 'On Hold' for each customer."})

[{'Answer': "'America West Airlines B757-200, 9,653', '2002 Chevy Corvette, 9,446', '1912 Ford Model T Delivery Wagon, 9,173'",
  'Question': "What are the top 3 products by quantity of on hand for products listed on 'On Hold' orders?",
  'SQLQuery': "SELECT productName, FORMAT(quantityInStock,0) AS `Quantity in stock` FROM OrderDetails JOIN Orders ON Orders.orderNumber = OrderDetails.orderNumber JOIN Products on OrderDetails.productCode = Products.productCode WHERE status = 'On Hold' order by quantityInStock desc limit 3;",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': "'Loui Bondur, 28,474', 'Larry Bott, 36,605', 'Pamela Castillo, 43,411', 'Julie Firrelli, 19,333','Andy Fixter, 28,129', 'Martin Gerard, 19,374'......",
  'Question': 'Compute the commission for each sales representative, assuming the commission is 5% of the value of an order. Sort by employee last name and first name',
  'SQLQuery': "SELECT CONCAT(firstName, ' ',lastName) AS Name, FORMAT(.05 * SUM(quantityOrde

In [14]:
example_selector.select_examples({"Question": "List all the products purchased by Herkku Gifts"})

[{'Answer': "'America West Airlines B757-200, 9,653', '2002 Chevy Corvette, 9,446', '1912 Ford Model T Delivery Wagon, 9,173'",
  'Question': "What are the top 3 products by quantity of on hand for products listed on 'On Hold' orders?",
  'SQLQuery': "SELECT productName, FORMAT(quantityInStock,0) AS `Quantity in stock` FROM OrderDetails JOIN Orders ON Orders.orderNumber = OrderDetails.orderNumber JOIN Products on OrderDetails.productCode = Products.productCode WHERE status = 'On Hold' order by quantityInStock desc limit 3;",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': 'Classic Cars, Vintage Cars, and Motorcycles',
  'Question': 'What are the top 3 most profitable product lines',
  'SQLQuery': 'SELECT productLine AS `Product line`, FORMAT(SUM(quantityOrdered*(priceEach -buyPrice)),0) AS Profit FROM Products JOIN OrderDetails ON Products.productCode = OrderDetails.productCode GROUP BY productLine ORDER BY SUM(quantityOrdered*(priceEach -buyPrice)) DESC LIMIT 3;',
  'SQLResult'

In [15]:
### my sql based instruction prompt
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [16]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


### Setting up PromptTemplete using input variables

In [17]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [18]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [19]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [20]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [24]:
new_chain("Find the products sold in 2003 but not 2004.")



> Entering new SQLDatabaseChain chain...
Find the products sold in 2003 but not 2004.
SQLQuery:SELECT DISTINCT productName FROM Products JOIN OrderDetails ON Products.productCode = OrderDetails.productCode JOIN Orders ON OrderDetails.orderNumber = Orders.orderNumber WHERE YEAR(Orders.orderDate) = 2003 AND Products.productCode NOT IN (SELECT productCode FROM OrderDetails JOIN Orders ON OrderDetails.orderNumber = Orders.orderNumber WHERE YEAR(Orders.orderDate) = 2004);
SQLResult: 
Answer:1952 Alpine Renault 1300, 1996 Moto Guzzi 1100i, 1993 Mazda RX-7, 1937 Lincoln Berline, 1936 Mercedes Benz 500k Roadster
> Finished chain.


{'query': 'Find the products sold in 2003 but not 2004.',
 'result': '1952 Alpine Renault 1300, 1996 Moto Guzzi 1100i, 1993 Mazda RX-7, 1937 Lincoln Berline, 1936 Mercedes Benz 500k Roadster'}

In [32]:
new_chain("What are the top 3 most profitable product lines and the profits calculated for each product line")



> Entering new SQLDatabaseChain chain...
What are the top 3 most profitable product lines and the profits calculated for each product line
SQLQuery:SELECT productLine AS `Product line`, FORMAT(SUM(quantityOrdered*(priceEach -buyPrice)),0) AS Profit FROM Products JOIN OrderDetails ON Products.productCode = OrderDetails.productCode GROUP BY productLine ORDER BY SUM(quantityOrdered*(priceEach -buyPrice)) DESC LIMIT 3;
SQLResult: [('Classic Cars', '1,526,212'), ('Vintage Cars', '737,268'), ('Motorcycles', '469,255')]
Answer:Classic Cars, Vintage Cars, and Motorcycles with profits of $1,526,212, $737,268, and $469,255 respectively.
> Finished chain.


{'query': 'What are the top 3 most profitable product lines and the profits calculated for each product line',
 'result': 'Classic Cars, Vintage Cars, and Motorcycles with profits of $1,526,212, $737,268, and $469,255 respectively.'}

In [41]:
new_chain("Display the difference in the amount received for each month of 2004 compared to 2003?")



> Entering new SQLDatabaseChain chain...
Display the difference in the amount received for each month of 2004 compared to 2003?
SQLQuery:SELECT FORMAT(SUM(amount),0) AS Difference FROM Payments WHERE YEAR(paymentDate) = 2004 GROUP BY MONTH(paymentDate) UNION SELECT FORMAT(SUM(amount),0) AS Difference FROM Payments WHERE YEAR(paymentDate) = 2003 GROUP BY MONTH(paymentDate);
SQLResult: [('185,103',), ('819,286',), ('378,094',), ('404,603',), ('857,187',), ('476,446',), ('234,152',), ('284,191',), ('208,524',), ('173,246',), ('106,652',), ('185,843',), ('180,219',), ('159,882',), ('144,384',), ('316,858',), ('136,314',), ('246,205',), ('694,293',), ('26,268',), ('826,638',), ('199,704',), ('158,247',), ('161,206',)]
Answer:The difference in the amount received for each month of 2004 compared to 2003 is: 185,103, 819,286, 378,094, 404,603, 857,187, 476,446, 234,152, 284,191, 208,524, 173,246, 106,652, 185,843, 180,219, 159,882, 144,384, 316,858, 136,314, 246,205, 694,293, 26,268, 826,638

{'query': 'Display the difference in the amount received for each month of 2004 compared to 2003?',
 'result': 'The difference in the amount received for each month of 2004 compared to 2003 is: 185,103, 819,286, 378,094, 404,603, 857,187, 476,446, 234,152, 284,191, 208,524, 173,246, 106,652, 185,843, 180,219, 159,882, 144,384, 316,858, 136,314, 246,205, 694,293, 26,268, 826,638, 199,704, 158,247, 161,206.'}

In [29]:
new_chain("What is the percentage value of each product in inventory sorted by the highest percentage. Display only the top 5")



> Entering new SQLDatabaseChain chain...
What is the percentage value of each product in inventory sorted by the highest percentage. Display only the top 5
SQLQuery:SELECT productName, FORMAT(quantityInStock/(SELECT SUM(quantityInStock) FROM Products)*100,2) AS `Percentage in stock` FROM Products ORDER BY `Percentage in stock` DESC LIMIT 5;
SQLResult: [('2002 Suzuki XREO', '1.80'), ('1995 Honda Civic', '1.76'), ('America West Airlines B757-200', '1.74'), ('2002 Chevy Corvette', '1.70'), ('1932 Model A Ford J-Coupe', '1.69')]
Answer:2002 Suzuki XREO, 1.80%; 1995 Honda Civic, 1.76%; America West Airlines B757-200, 1.74%; 2002 Chevy Corvette, 1.70%; 1932 Model A Ford J-Coupe, 1.69%
> Finished chain.


{'query': 'What is the percentage value of each product in inventory sorted by the highest percentage. Display only the top 5',
 'result': '2002 Suzuki XREO, 1.80%; 1995 Honda Civic, 1.76%; America West Airlines B757-200, 1.74%; 2002 Chevy Corvette, 1.70%; 1932 Model A Ford J-Coupe, 1.69%'}

In [42]:
new_chain.run("""WITH
t2003 AS (SELECT YEAR(paymentDate) AS 'year', MONTH(paymentDate) AS 'month', sum(amount) AS amount FROM Payments
WHERE YEAR(paymentDate) = 2003
GROUP BY YEAR(paymentDate), MONTH(paymentDate)),
t2004 AS (SELECT YEAR(paymentDate) AS 'year', MONTH(paymentDate) AS 'month', sum(amount) AS amount FROM Payments
WHERE YEAR(paymentDate) = 2004
GROUP BY YEAR(paymentDate), MONTH(paymentDate))
SELECT t2003.month,format((t2004.amount - t2003.amount),2) AS variance
FROM t2003 JOIN t2004
ON t2003.month = t2004.month
ORDER BY t2003.month;""")



> Entering new SQLDatabaseChain chain...
WITH
t2003 AS (SELECT YEAR(paymentDate) AS 'year', MONTH(paymentDate) AS 'month', sum(amount) AS amount FROM Payments
WHERE YEAR(paymentDate) = 2003
GROUP BY YEAR(paymentDate), MONTH(paymentDate)),
t2004 AS (SELECT YEAR(paymentDate) AS 'year', MONTH(paymentDate) AS 'month', sum(amount) AS amount FROM Payments
WHERE YEAR(paymentDate) = 2004
GROUP BY YEAR(paymentDate), MONTH(paymentDate))
SELECT t2003.month,format((t2004.amount - t2003.amount),2) AS variance
FROM t2003 JOIN t2004
ON t2003.month = t2004.month
ORDER BY t2003.month;
SQLQuery:SELECT t2003.month, FORMAT((t2004.amount - t2003.amount), 2) AS variance
FROM (SELECT YEAR(paymentDate) AS 'year', MONTH(paymentDate) AS 'month', SUM(amount) AS amount FROM Payments
WHERE YEAR(paymentDate) = 2003
GROUP BY YEAR(paymentDate), MONTH(paymentDate)) AS t2003
JOIN (SELECT YEAR(paymentDate) AS 'year', MONTH(paymentDate) AS 'month', SUM(amount) AS amount FROM Payments
WHERE YEAR(paymentDate) = 2004
GROU

'The variance in payments between 2003 and 2004 for each month, sorted by month.'